In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_36/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Modelos de Ensamble

## Introducción

Usando el dataset Movie_classification, que vimos en la clase de CART, vamos a 

* Entrenar tres modelos base de clasificación, y construir a partir de ellos un ensamble para predecir el valor de la variable Start_Tech_Oscar usando predict_proba y AUC como métrica

* Comparar la performance de los modelos base con la del modelo de ensamble, y con el resultado del checkpoint.


## Dataset

En esta clase usaremos un dataset con info de películas ("Movie_classification.csv").  
Este dataset esta conformado por los siguientes features:  

 *   **Marketing expense:**    (float64)    Gasto total en Marketing      
 *   **Production expense:**   (float64)    Gasto total de Producción
 *   **Multiplex coverage:**   (float64)    Cobertura promedio de Multiplex
 *   **Budget:**               (float64)    Presupuesto
 *   **Movie_length:**         (float64)    Duración de la película
 *   **Lead_ Actor_Rating:**   (float64)    Puntaje sobre el actor principal
 *   **Lead_Actress_rating:**  (float64)    Puntaje sobre la actriz principal
 *   **Director_rating:**      (float64)    Puntaje sobre el Director
 *   **Producer_rating:**      (float64)    Puntaje sobre el Productor
 *   **Critic_rating:**        (float64)    Puntaje que le puso la crítica
 *   **Trailer_views:**        (int64)      Cantidad de vistas del Trailer
 *   **3D_available:**         (object)     Si esta disponible en 3D (Yes/No)
 *   **Time_taken:**           (float64)    Duración de la película
 *   **Twitter_hastags:**      (float64)    Cantidad de menciones en twitter
 *   **Genre:**                (object)     Genero de la película
 *   **Avg_age_actors:**       (int64)      Edad promedio de los actores
 *   **Num_multiplex:**        (int64)      Cantidad de Multiplex
 *   **Collection:**           (int64)      Recaudación
 *   **Start_Tech_Oscar:**     (int64)      Si recibió un oscar o no.
 

## Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

## Ejercicio 1

Leamos el dataset "Movie_classification", eliminemos los registros con valores faltantes y las features categóricas "Genre" y "3D_available"

In [3]:
df = pd.read_csv('../Data/Movie_classification.csv')

In [4]:
df.head()

,Marketing expense,Production expense,Multiplex coverage,Budget,Movie_length,Lead_ Actor_Rating,Lead_Actress_rating,Director_rating,Producer_rating,Critic_rating,Trailer_views,3D_available,Time_taken,Twitter_hastags,Genre,Avg_age_actors,Num_multiplex,Collection,Start_Tech_Oscar
0,20.1264,59.62,0.462,36524.125,138.7,7.825,8.095,7.910,7.995,7.94,527367,YES,109.60,223.840,Thriller,23,494,48000,1
1,20.5462,69.14,0.531,35668.655,152.4,7.505,7.650,7.440,7.470,7.44,494055,NO,146.64,243.456,Drama,42,462,43200,0
2,20.5458,69.14,0.531,39912.675,134.6,7.485,7.570,7.495,7.515,7.44,547051,NO,147.88,2022.400,Comedy,38,458,69400,1
3,20.6474,59.36,0.542,38873.890,119.3,6.895,7.035,6.920,7.020,8.26,516279,YES,185.36,225.344,Drama,45,472,66800,1
4,21.3810,59.36,0.542,39701.585,127.7,6.920,7.070,6.815,7.070,8.26,531448,NO,176.48,225.792,Drama,55,395,72400,1


In [6]:
df.drop(["Genre","3D_available"], axis=1, inplace= True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 505
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Marketing expense    494 non-null    float64
 1   Production expense   494 non-null    float64
 2   Multiplex coverage   494 non-null    float64
 3   Budget               494 non-null    float64
 4   Movie_length         494 non-null    float64
 5   Lead_ Actor_Rating   494 non-null    float64
 6   Lead_Actress_rating  494 non-null    float64
 7   Director_rating      494 non-null    float64
 8   Producer_rating      494 non-null    float64
 9   Critic_rating        494 non-null    float64
 10  Trailer_views        494 non-null    int64  
 11  Time_taken           494 non-null    float64
 12  Twitter_hastags      494 non-null    float64
 13  Avg_age_actors       494 non-null    int64  
 14  Num_multiplex        494 non-null    int64  
 15  Collection           494 non-null    int

In [9]:
df.dropna(inplace= True)

## Ejercicio 2

Crear los conjuntos de train y test

In [12]:
X = df.drop('Start_Tech_Oscar', axis=1)

y= df.Start_Tech_Oscar

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 1)

## Ejercicio 3

Estandarizar las features para los conjuntos de train y test

In [13]:
esc = StandardScaler()
X_train = esc.fit_transform(X_train)
X_test = esc.transform(X_test)

## Ejercicio 4

Entrenar un modelo Naive Bayes para predecir el valor de "Start_Tech_Oscar".

Evaluar su perfomance en test mediante AUC.


In [14]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)


GaussianNB()

In [16]:
from sklearn.metrics import roc_auc_score

auc_gnb = roc_auc_score(y_test, gnb.predict(X_test))
auc_gnb

0.551470588235294

## Ejercicio 5

Entrenar un modelo KNN para predecir el valor de "Start_Tech_Oscar".

Evaluar su perfomance en test mediante AUC.


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [34]:
param = {'n_neighbors': range(5,30),
             'weights':['uniform','distance'],
             'p':[1,2]}

kfolds=KFold(n_splits=5, shuffle=True, random_state=1)
knn = KNeighborsClassifier()

gs = GridSearchCV(estimator =knn, param_grid=param, cv=kfolds, scoring='f1', n_jobs=-1)



In [35]:
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 30), 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1')

In [37]:
knn=gs.best_estimator_

In [38]:
roc_auc_score(y_test, knn.predict(X_test))

0.6165966386554621

## Ejercicio 6

Entrenar unárbol de clasificación para predecir el valor de "Start_Tech_Oscar".

Evaluar su perfomance en test mediante la matriz de confusión.


In [42]:
arbol=tree.DecisionTreeClassifier()
arbol.fit(X_train, y_train)

roc_auc_score(y_test, arbol.predict(X_test))

0.6228991596638654

In [43]:
confusion_matrix(y_test, arbol.predict(X_test))

array([[36, 20],
       [27, 41]])

## Ejercicio 7


Construir un modelo de ensamble usando como modelos base los tres modelos anteriores.

Para esto, escribir la función `predict_proba_ensamble` que calcule la probabilidad de 1 como el promedio de las probabilidades respuesta de los predictores base

Evaluar la performance del ensamble (usando como umbral 0.5) mediante

* AUC

* accuracy

* confusion_matrix


Comparar estos resultados con los obtenidos en el ejercicio 7 del checkpoint.


In [44]:

from sklearn.ensemble import VotingClassifier

In [45]:
vc = VotingClassifier([('gnb',gnb),('knn',knn),('arbolito',arbol)],voting='soft')

In [46]:
vc.fit(X_train, y_train)

VotingClassifier(estimators=[('gnb', GaussianNB()),
                             ('knn',
                              KNeighborsClassifier(n_neighbors=16, p=1,
                                                   weights='distance')),
                             ('arbolito', DecisionTreeClassifier())],
                 voting='soft')

In [47]:
roc_auc_score(y_test, vc.predict(X_test))

0.6129201680672267

In [54]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

accuracy_score(y_test, vc.predict(X_test))

0.6048387096774194

In [55]:
confusion_matrix(y_test, vc.predict(X_test))

array([[39, 17],
       [32, 36]])

## Opcional:

¿Pueden mejorar el valor de accuracy del ensamble usando otro valor (distinto de 0.5) para el umbral? 

¿Qué pasa con la matriz de confusión cuando mejora el valor de accuracy?

Pueden hacer la prueba en un ciclo for con distintos valores de threshold, o usar roc_curve (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) para intentar responder.